# Setup: Connectivity

In [1]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


My Client: <katana_enterprise.remote.sync_wrappers.Client object at 0x7f9c228dfbb0>
My Graph:  <_Graph my_graph, 9XsFVxEtTWkJCWAKvVVGUuAzf2J6xyG22QMsWB1WYB9a, 5>



#  Step 01: Enter a new Medical Description

In [2]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


Textarea(value='Odd mole on my ear', description='Enter Text: ', layout=Layout(flex='0 1 auto', height='100px'…

In [3]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Ptient Vist Id: " % (l_pvid))
print("")
print("Medical Description encoding:")
print(l_return)


          0/? [?op/s]


Host 0 errors:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 86, in execute
    value = function(graph)
  File "/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/dataframe_importer/__init__.py", line 453, in remote_write_intermediate_parquet_files
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/dataframe_importer/__init__.py", line 207, in _write_intermediate_parquet_files
    node_header = "\n".join(_write_dfs_to_parquet(node_df_infos, "nodes"))
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/dataframe_importer/__init__.py", line 201, in _write_dfs_to_parquet
    dask.dataframe.to_parquet(
  File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py", line 963, in to_parquet
    out = out.compute(**compute_kwargs)
  File "/opt/miniconda/lib/python3.8/site-packages/dask/base.py", line 315, in compute
    (result

ValueError: Failed to convert partition to expected pyarrow schema:
    `ArrowTypeError("Expected bytes, got a 'list' object", 'Conversion failed for column transcription with type object')`

Expected partition schema:
    id: string
    transcription: string

Received partition schema:
    id: string
    transcription: list<item: struct<certaintyAssessment: struct<confidence: double, value: string>, confidence: double, entityId: string, linkedEntities: list<item: struct<entityId: string>>, mentionId: string, preferredTerm: string, subject: struct<confidence: double, value: string>, temporalAssessment: struct<confidence: double, value: string>, text: struct<beginOffset: int64, content: string>, type: string, vocabularyCodes: list<item: string>>>
      child 0, item: struct<certaintyAssessment: struct<confidence: double, value: string>, confidence: double, entityId: string, linkedEntities: list<item: struct<entityId: string>>, mentionId: string, preferredTerm: string, subject: struct<co

# Step 02:  Examine the Graph

In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="sample_name"      ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary),
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, ms, ios
   
   """.format(l_pvid)

l_result = my_graph.query(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


In [ ]:

#  Sample data from above,
#
#     200
#     
#     {
#        "entityMentions": [
#           {
#              "mentionId": "1",
#              "type": "MEDICINE",
#              "text": {
#                 "content": "Insulin regimen",
#                 "beginOffset": 0
#              },
#              "linkedEntities": [
#                 {
#                    "entityId": "UMLS/C0021641"
#                 },
#                 {
#                    "entityId": "UMLS/C0795635"
#                 },
#                 {
#                    "entityId": "UMLS/C1533581"
#                 },
#                 {
#                    "entityId": "UMLS/C3537244"
#                 },
#                 {
#                    "entityId": "UMLS/C3714501"
#                 }
#              ],
#              "temporalAssessment": {
#                 "value": "CURRENT",
#                 "confidence": 0.8573660850524902
#              },
#              "certaintyAssessment": {
#                 "value": "LIKELY",
#                 "confidence": 0.9751282930374146
#              },
#              "subject": {
#                 "value": "PATIENT",
#                 "confidence": 0.9995787739753723
#              },
#              "confidence": 0.6379408836364746
#           },
#           {
#              "mentionId": "2",
#              "type": "MED_DOSE",
#              "text": {
#                 "content": "5 units",
#                 "beginOffset": 22
#              },
#              "confidence": 0.7443782091140747
#           },
#           {
#              "mentionId": "3",
#              "type": "MED_ROUTE",
#              "text": {
#                 "content": "IV",
#                 "beginOffset": 30
#              },
#              "linkedEntities": [
#                 {
#                    "entityId": "UMLS/C0348016"
#                 }
#              ],
#              "confidence": 0.779011607170105
#           }
#        ],
#        "entities": [
#           {
#              "entityId": "UMLS/C0021641",
#              "preferredTerm": "Insulin",
#              "vocabularyCodes": [
#                 "FMA/83365",
#                 "LNC/LA15805-7",
#                 "LNC/LP14676-8",
#                 "LNC/LP16325-0",
#                 "LNC/LP32542-0",
#                 "LNC/LP70329-5",
#                 "LNC/MTHU002108",
#                 "LNC/MTHU019392",
#                 "MSH/D007328",
#                 "MTH/NOCODE"
#              ]
#           },
#           {
#              "entityId": "UMLS/C0348016",
#              "preferredTerm": "Intravenous",
#              "vocabularyCodes": [
#                 "LNC/LA9437-0",
#                 "LNC/LP32453-0",
#                 "MTH/NOCODE",
#                 "NCI/C13346"
#              ]
#           },
#           {
#              "entityId": "UMLS/C0795635",
#              "preferredTerm": "insulin, regular, human",
#              "vocabularyCodes": [
#                 "LNC/LP17001-6",
#                 "MSH/D061386",
#                 "MTH/NOCODE",
#                 "NCI/C29125",
#                 "RXNORM/253182",
#                 "VANDF/4017559",
#                 "VANDF/4017569",
#                 "VANDF/4019786"
#              ]
#           },
#           {
#              "entityId": "UMLS/C1533581",
#              "preferredTerm": "Therapeutic Insulin",
#              "vocabularyCodes": [
#                 "MTH/NOCODE",
#                 "NCI/C581"
#              ]
#           },
#           {
#              "entityId": "UMLS/C3537244",
#              "preferredTerm": "Insulins",
#              "vocabularyCodes": [
#                 "MSH/D061385",
#                 "MTH/NOCODE"
#              ]
#           },
#           {
#              "entityId": "UMLS/C3714501",
#              "preferredTerm": "Insulin Drug Class",
#              "vocabularyCodes": [
#                 "MTH/NOCODE",
#                 "VANDF/4021631"
#              ]
#           }
#        ],
#        "relationships": [
#           {
#              "subjectId": "1",
#              "objectId": "2",
#              "confidence": 0.9996469616889954
#           },
#           {
#              "subjectId": "1",
#              "objectId": "3",
#              "confidence": 0.9995671510696411
#           }
#        ]
#     }
